In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
import pickle

In [9]:
import tensorflow as tf
print(tf.__version__)


2.20.0


In [11]:
# ---------------- Helper Functions ----------------
R = 6371  # Earth’s radius in km

def deg_to_rad(degrees):
    return degrees * (np.pi / 180)

def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2 - lat1)
    d_lon = deg_to_rad(lon2 - lon1)
    a = np.sin(d_lat / 2) ** 2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# ---------------- Load & Clean Data ----------------
data = pd.read_csv("train.csv")

In [13]:
# Clean target column
data['Time_taken(min)'] = data['Time_taken(min)'].astype(str).str.extract(r'(\d+\.?\d*)')[0]
data['Time_taken(min)'] = pd.to_numeric(data['Time_taken(min)'], errors='coerce')

# Convert numeric columns
numeric_columns = ['Delivery_person_Age', 'Delivery_person_Ratings', 
                   'Restaurant_latitude', 'Restaurant_longitude', 
                   'Delivery_location_latitude', 'Delivery_location_longitude']


In [15]:
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

data_clean = data.dropna(subset=numeric_columns + ['Time_taken(min)'], how='any').copy()

data_clean['distance'] = data_clean.apply(
    lambda row: distcalculate(
        row['Restaurant_latitude'],
        row['Restaurant_longitude'],
        row['Delivery_location_latitude'],
        row['Delivery_location_longitude']
    ), axis=1
)


In [21]:
# ---------------- Train Model ----------------
X = np.array(data_clean[["Delivery_person_Age", "Delivery_person_Ratings", "distance"]])
y = np.array(data_clean[["Time_taken(min)"]])

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.1, random_state=42)

# Reshape for LSTM
xtrain = xtrain.reshape((xtrain.shape[0], xtrain.shape[1], 1))
xtest = xtest.reshape((xtest.shape[0], xtest.shape[1], 1))

from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

# Build LSTM
model = Sequential()
model.add(Input(shape=(xtrain.shape[1], 1)))   # Explicit Input layer
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(xtrain, ytrain, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
1229/1229 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 105.0108
Epoch 2/5
1229/1229 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 63.4611
Epoch 3/5
1229/1229 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 60.3375
Epoch 4/5
1229/1229 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 59.0714
Epoch 5/5
1229/1229 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 58.2773


In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# ---------------- Evaluate Model ----------------
# Predict on test set
y_pred = model.predict(xtest)

# Flatten arrays
ytest_flat = ytest.flatten()
y_pred_flat = y_pred.flatten()

# Metrics
mae = mean_absolute_error(ytest_flat, y_pred_flat)
mse = mean_squared_error(ytest_flat, y_pred_flat)
rmse = np.sqrt(mse)
r2 = r2_score(ytest_flat, y_pred_flat)

print("📊 Model Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
📊 Model Evaluation Metrics:
Mean Absolute Error (MAE): 6.17
Mean Squared Error (MSE): 60.99
Root Mean Squared Error (RMSE): 7.81
R² Score: 0.30


In [25]:
with open("food_delivery_model.pkl", "wb") as f:
    pickle.dump(model, f)


In [27]:
from tensorflow.keras.models import save_model
model.save("food_delivery_model.keras")   # or .keras
